# TECHNIKI PROGRAMOWANIA - projekt 3
### Michał Mańkowski 197622
### Kacper Kobrzyński 197668

Wykorzystane biblioteki:
- matplotplusplus (c++)
- pybind11 (c++)

Obie są załączone jako submodule

Przy budowie proszę umieścić biblioteke w folderze build


# Wizualizacja sygnałów 1D
Funkcja visualize_signal(signal, time, filename) Funkcja zapisuje wykres sygnału w pliku podanym przez nas. Sygnał powinen być listą floatów lub listą typu automatycznie konwertowanego na float przez pythona, ale tak będzie dla każdego typu więc nie ma sensu pisać tego za każdym razem. Time to float i filename to string. 
Do zaprezentowania wczytujemy plik dźwiękowy jako sygnał i odczytujemy jego czas trwania w sekundach używając wbudowanej w pythonie biblioteki wave:

In [2]:
import wave
import struct
import build.siganalysis as sig

def read_wave(filename: str) -> tuple[float, list[float]]:
    with wave.open(filename, 'rb') as f:
        data = f.readframes(f.getnframes())
        duration = f.getnframes() / f.getframerate()
        samples = struct.unpack('{n}h'.format(n=f.getnframes()*f.getnchannels()), data)
        return duration, samples

audio = read_wave("dc.wav")
sig.visualize_signal(audio[1], audio[0], "raport_img/visualize_audio.jpg")

         line 91400: warning: refresh not possible and replot is disabled
         line 91400: warning: refresh not possible and replot is disabled
         line 91400: warning: refresh not possible and replot is disabled
         line 91400: warning: refresh not possible and replot is disabled
         line 91400: warning: refresh not possible and replot is disabled
         line 91400: warning: refresh not possible and replot is disabled
         line 91400: warning: refresh not possible and replot is disabled
         line 91400: warning: refresh not possible and replot is disabled
         line 91400: warning: refresh not possible and replot is disabled
         line 91400: warning: refresh not possible and replot is disabled
         line 91400: warning: refresh not possible and replot is disabled
         line 91400: warning: refresh not possible and replot is disabled


![Visualized Audio](raport_img/visualize_audio.jpg)

# Generowanie sygnałów
generate_signal(signal_type, time, sampling, amplitude, frequency) Funkcja zwraca liste floatów, signal_type to enum, który ma 4 różne opcje:
- SignalTypes.sig_sin
- SignalTypes.sig_cos
- SignalTypes.sig_square
- SignalTypes.sig_sawtooth
Każda odzwierciedla typ generowanego sygnału.
time, amplitude i frequency to float, a sampling to int. Frequency nie jest wartością w Hz, tylko częstotliwością kątową.
Przykłady użycia przy jednoczesnej wizualizacji używając wcześniej przedstawionej funkcji:

In [9]:
Signal = sig.generate_signal(sig.SignalTypes.sig_sin, 10, 10000, 1, 1)
sig.visualize_signal(Signal, 10, "raport_img/visualize_sig_sin1.jpg")

![Sig Sin1](raport_img/visualize_sig_sin1.jpg)

In [10]:
Signal = sig.generate_signal(sig.SignalTypes.sig_cos, 10, 10000, 1, 1)
sig.visualize_signal(Signal, 10, "raport_img/visualize_sig_cos1.jpg")

![Cosine plot1](raport_img/visualize_sig_cos1.jpg)

In [11]:
Signal = sig.generate_signal(sig.SignalTypes.sig_square, 10, 10000, 1, 1)
sig.visualize_signal(Signal, 10, "raport_img/visualize_sig_square1.jpg")

![Square Signal](raport_img/visualize_sig_square1.jpg)

In [12]:
Signal = sig.generate_signal(sig.SignalTypes.sig_sawtooth, 10, 10000, 1, 1)
sig.visualize_signal(Signal, 10, "raport_img/visualize_sig_sawtooth1.jpg")

![Signal Sawtooth1](raport_img/visualize_sig_sawtooth1.jpg)

In [19]:
Signal = sig.generate_signal(sig.SignalTypes.sig_sin, 10, 10000, 25, 5)
sig.visualize_signal(Signal, 10, "raport_img/visualize_sig_sin2.jpg")

![Signal Sine2](raport_img/visualize_sig_sin2.jpg)

# DFT
dft(signal) Zależnie od tego czy zostanie podana lista floatów, czy lista liczb zespolonych mamy różne wyniki.
Jeśli podany sygnał jest listą floatów to zostaje zwrócona lista liczb zespolonych,
Jesli podany sygnał jest listą liczb zespolonych to zostaje zwrócona lista floatów.
W taki sposób dft jest funkcją, wykonującą dft i ją odwracającą.
Do prezentacji działania zostanie użyta jeszcze funkcja complexToPowerVector(complex_list) zwracająca listę mocy liczb zespolonych.

In [26]:
SignalAfterDFT = sig.dft(audio[1])
SignalPower = sig.complexToPowerVector(Signal)
sig.visualize_signal(SignalPower, 420, "raport_img/dft_audio.jpg")


![DFT Audio1](raport_img/dft_audio.jpg)

Odwracamy DFT:

In [27]:
SignalReversedDFT = sig.dft(SignalAfterDFT)
sig.visualize_signal(SignalReversedDFT, audio[0], "raport_img/audio_reversed_dft.jpg")

![Audio reversed dft](raport_img/audio_reversed_dft.jpg)

Jak widać sygnał pozostaje niezmieniony.

Ostatnią funkcja jaka została to frequency_filter(signal, cutoff) gdzie signal to lista floatów, a cutoff to float.

Funkcja ta dzięki dft filture częstotliwości mniejsze niż podany przez nasz cutoff.

In [30]:
FilteredSignal = sig.frequency_filter(audio[1], 300)

def write_wave(filename: str, signal: list, duration: int):
    channels = 2
    framerate = int(len(signal) / (duration * channels)) 
    samplewidth = 2
    with wave.open(filename, 'wb') as f:
        f.setnchannels(channels)
        f.setsampwidth(samplewidth)
        f.setframerate(framerate)
        signal_bytes = struct.pack('<' + 'h' * len(signal), *[int(sample) for sample in signal])
        f.writeframes(signal_bytes)

write_wave("dc_filtered.wav", FilteredSignal, audio[0])
sig.visualize_signal(FilteredSignal, audio[0], "raport_img/filtered_audio.jpg")

Odsłuchać różnicy między oryginałem i po filtracji można odsłuchać. Oryginał nazywa się dc.wav, a po filtracji dc_filtered.wav.
Można też zobaczyć różnicę wizualnie:

![Audio filtered](raport_img/filtered_audio.jpg)